<a href="https://colab.research.google.com/github/omidm7/calciumImagingAnalysis/blob/master/Excel_Duplication_Finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Analyze the sheets in the file "duplicate_data.xlsx" to identify duplicate cell values, ignoring zero and empty values. Highlight cells with values duplicated within the same sheet in yellow and cells with values duplicated across different sheets in green.

## Load the excel file

### Subtask:
Read the data from the provided Excel file into a Pandas DataFrame, handling multiple sheets.


**Reasoning**:
Import the pandas library and read the Excel file into a dictionary of DataFrames, handling multiple sheets by using `sheet_name=None`.



In [ ]:
import pandas as pd

excel_data = pd.read_excel('/content/miry_practice.xlsx', sheet_name=None)

## Identify intra-sheet duplicates

### Subtask:
For each sheet, identify cells with duplicate values within that sheet, ignoring zero and empty values. Store the locations (sheet name and cell address) of these duplicates.


**Reasoning**:
Initialize the dictionary to store intra-sheet duplicates and iterate through each sheet and column to find and store the locations of duplicate values, ignoring zero and empty values.



In [ ]:
intra_sheet_duplicates = {}

for sheet_name, df in excel_data.items():
    intra_sheet_duplicates[sheet_name] = []
    for col in df.columns:
        # Identify duplicated values in the current column, ignoring zero and empty values
        duplicated_values = df[col][(df[col].duplicated(keep=False)) & (df[col] != 0) & (df[col].notna())]

        # Find all cell addresses for each duplicated value
        for value in duplicated_values.unique():
            cell_locations = df[(df[col] == value) & (df[col] != 0) & (df[col].notna())].index.tolist()
            for row_index in cell_locations:
                intra_sheet_duplicates[sheet_name].append(f'{chr(65 + df.columns.get_loc(col))}{row_index + 1}')


## Identify inter-sheet duplicates

### Subtask:
Compare the values across all sheets to identify cells with duplicate values in different sheets, ignoring zero and empty values. Store the locations (sheet names and cell addresses) of these duplicates.


**Reasoning**:
Initialize an empty dictionary, create a list of all values with their locations, iterate through the list to find inter-sheet duplicates, and store the locations of these duplicates in the dictionary.



In [ ]:
inter_sheet_duplicates = {}
all_values_with_locations = []

for sheet_name, df in excel_data.items():
    for col_index, col in enumerate(df.columns):
        for row_index, value in df[col].items():
            if pd.notna(value) and value != 0:
                all_values_with_locations.append({
                    'value': value,
                    'sheet': sheet_name,
                    'cell': f'{chr(65 + col_index)}{row_index + 1}'
                })

from collections import defaultdict
value_locations = defaultdict(list)
for item in all_values_with_locations:
    value_locations[item['value']].append({'sheet': item['sheet'], 'cell': item['cell']})

inter_sheet_duplicates_raw = {}
for value, locations in value_locations.items():
    sheets_with_value = set([loc['sheet'] for loc in locations])
    if len(sheets_with_value) > 1:
        for loc in locations:
            if loc['sheet'] not in inter_sheet_duplicates_raw:
                inter_sheet_duplicates_raw[loc['sheet']] = []
            inter_sheet_duplicates_raw[loc['sheet']].append(loc['cell'])

for sheet_name, cell_list in inter_sheet_duplicates_raw.items():
    inter_sheet_duplicates[sheet_name] = list(set(cell_list))

display(inter_sheet_duplicates)

## Prioritize highlighting

### Subtask:
Determine which duplicates are both intra-sheet and inter-sheet. Inter-sheet duplicates should be prioritized for green highlighting, and intra-sheet duplicates (that are not inter-sheet duplicates) should be highlighted yellow.


**Reasoning**:
Initialize empty dictionaries and iterate through the intra-sheet duplicates to identify those that are not inter-sheet duplicates, adding them to the highlight_yellow dictionary. Then, iterate through the inter-sheet duplicates and add all of them to the highlight_green dictionary. Finally, remove any cell locations from highlight_yellow that are also in highlight_green.



In [ ]:
highlight_yellow = {}
highlight_green = {}

for sheet_name, cell_list in intra_sheet_duplicates.items():
    highlight_yellow[sheet_name] = []
    for cell in cell_list:
        if sheet_name in inter_sheet_duplicates and cell not in inter_sheet_duplicates[sheet_name]:
            highlight_yellow[sheet_name].append(cell)
        elif sheet_name not in inter_sheet_duplicates:
             highlight_yellow[sheet_name].append(cell)


for sheet_name, cell_list in inter_sheet_duplicates.items():
    highlight_green[sheet_name] = cell_list

for sheet_name, cell_list in highlight_green.items():
    if sheet_name in highlight_yellow:
        highlight_yellow[sheet_name] = [cell for cell in highlight_yellow[sheet_name] if cell not in cell_list]

display(highlight_yellow)
display(highlight_green)

## Apply highlighting

### Subtask:
Create a new Excel file or modify the existing one to apply the specified background colors (yellow for intra-sheet duplicates and green for inter-sheet duplicates) to the identified cells.


**Reasoning**:
Import the openpyxl library and load the existing Excel workbook. Then iterate through the highlight_yellow and highlight_green dictionaries to color the cells accordingly. Finally, save the modified workbook to a new Excel file.



In [ ]:
import openpyxl
from openpyxl.styles import PatternFill

# Load the existing Excel workbook
workbook = openpyxl.load_workbook('/content/miry_practice.xlsx')

yellow_fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
green_fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')

for sheet_name, cells in highlight_yellow.items():
    if sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        for cell in cells:
            sheet[cell].fill = yellow_fill

for sheet_name, cells in highlight_green.items():
    if sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        for cell in cells:
            sheet[cell].fill = green_fill

workbook.save('highlighted_duplicates.xlsx')

## Summary:

### Data Analysis Key Findings

*   The analysis identified cells with duplicate values within the same sheet, excluding zeros and empty values.
*   The analysis also identified cells with duplicate values across different sheets, excluding zeros and empty values.
*   A prioritization was established: cells that were both intra-sheet and inter-sheet duplicates were categorized as inter-sheet duplicates for highlighting purposes.
*   A new Excel file named 'highlighted\_duplicates.xlsx' was created with intra-sheet duplicates (not also inter-sheet) highlighted in yellow and inter-sheet duplicates highlighted in green.

### Insights or Next Steps

*   The generated 'highlighted\_duplicates.xlsx' file provides a clear visual representation of duplicate data for review and potential cleaning.
*   The identified duplicate locations can be used for further data cleaning steps, such as removing or consolidating duplicate entries.
